In [1]:
import time
import optax
import haiku as hk
import jax.numpy as jnp
import jax
from torch.utils import data
import numpy as np
from data_utils import SATTrainingDataset, JraphDataLoader
from model import network_definition

NUM_EPOCHS = 10
f=0.0001

# # Make a batched version of the forwarding
# batched_predict = jax.vmap(network.apply, in_axes=(None, 0))


# def loss(params, problems, targets):
#     preds = batched_predict(params, problems)
#     return -jnp.mean(preds * targets)


def one_hot(x, k, dtype=jnp.float32):
    """Create a one-hot encoding of x of size k."""
    return jnp.array(x[:, None] == jnp.arange(k), dtype)

vmap_one_hot=jax.vmap(one_hot, in_axes=(0,None), out_axes=0)

path='/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/BroadcastTestSet_subset/'
sat_data = SATTrainingDataset(path)
train_data, test_data = data.random_split(sat_data, [0.8, 0.2])

train_loader = JraphDataLoader(train_data, batch_size=1, shuffle=True)

network = hk.without_apply_rng(hk.transform(network_definition))
params = network.init(jax.random.PRNGKey(42), sat_data[0][0].graph)

opt_init, opt_update = optax.adam(1e-3)
opt_state = opt_init(params)





2022-12-20 14:52:32.873166: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2022-12-20 14:52:33.624153: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 1.751174s
Constant folding an instruction is taking > 1s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necess

In [2]:
@jax.jit
def compute_log_probs(decoded_nodes, mask, candidate):
            a=jax.nn.log_softmax(decoded_nodes) * mask[:, None]
            b= jnp.dot(candidate,a.T)
            return b


vmap_compute_log_probs=jax.vmap(compute_log_probs, in_axes=(None,None, 0), out_axes=0)



@jax.jit
def update(params, opt_state, x, y, f):
            batch_masks=x[0]
            batch_graphs=x[1]
            batch_c=y[0]
            batch_e=y[1]
            #print(len(batch_masks))
            #print(len(batch_graphs))
            #print(len(batch_c))
            #print(len(batch_e))
            #print(batch_e[0])
            #print(batch_e)
            #loss=new_prediction_loss(params, batch_masks[0], batch_graphs[0], batch_c[0], batch_e[0], f)
            #print(loss)
            g=jax.grad(new_prediction_loss)(params, batch_masks[0], batch_graphs[0], batch_c[0], batch_e[0], f)
            #g=jax.grad(batched_loss)(params, *x, c, e, f)
            ####
            updates, opt_state = opt_update(g, opt_state)
            return optax.apply_updates(params, updates), opt_state


@jax.jit
def prediction_loss(params, mask, graph, solution):
        decoded_nodes = network.apply(params, graph)
        solution = one_hot(solution, 2)
        # We interpret the decoded nodes as a pair of logits for each node.
        log_prob = jax.nn.log_softmax(decoded_nodes) * solution
        return -jnp.sum(log_prob * mask[:, None]) / jnp.sum(mask)


@jax.jit
def new_prediction_loss(params, mask, graph, candidates, energies, f: float):
            decoded_nodes = network.apply(params, graph)
            candidates = vmap_one_hot(candidates, 2)
            log_prob=vmap_compute_log_probs(decoded_nodes, mask, candidates)
            weights = jax.nn.softmax(- f * energies)
            #print(np.shape(weights))
            weighted_log_probs = jax.vmap(jnp.dot,axis_name=(0,0), out_axes=0)(log_prob, weights)
            #print(np.shape(weighted_log_probs))
            summed_weighted_log_probs=jnp.sum(weighted_log_probs, axis=0) #sum over all candidates
            #print(np.shape(summed_weighted_log_probs))
            #loss=1
            print(np.shape(summed_weighted_log_probs))
            print(np.shape(mask[:,None]))
            a=jnp.dot(summed_weighted_log_probs ,mask[:, None])
            b=jnp.asarray(a)
            print(b)
            loss=-np.sum(b) #/ jnp.sum(mask)
            print(loss)
            #print(np.shape(loss))
            #print(loss)
            return loss#summed_weighted_log_probs#loss

#batched_loss = jax.vmap(new_prediction_loss_single, in_axes=(None, None, None, 1,1, None), out_axes=0)

In [3]:
import scipy
counter=0
for (batch_p, batch_ce) in train_loader:
            counter=counter+1
            print("batch_number", counter)
            batch_masks=batch_p[0]
            batch_graphs=batch_p[1]
            batch_c=batch_ce[0]
            batch_e=batch_ce[1]
            #print(len(batch_masks))
            #print(len(batch_graphs))
            #print(len(batch_c))
            #print(len(batch_e))
            #print(batch_e)
            mask = batch_masks[0]
            graph = batch_graphs[0]
            c = batch_c[0]
            e= batch_e[0]
            print(e)
            f=0.0001
            decoded_nodes = network.apply(params, graph)
            candidates = vmap_one_hot(c, 2)
            #log_prob=compute_log_probs(decoded_nodes, mask, candidates[0])
            #log_prob= jax.vmap(compute_log_probs, in_axes=(None,None, 0), out_axes=0)(decoded_nodes, mask, candidates)
            log_prob=vmap_compute_log_probs(decoded_nodes, mask, candidates)
            weights = jax.nn.softmax(- f * e)
            #weights= scipy.special.softmax(-f*e)
            print(weights)
            #print(np.shape(weights))
            weighted_log_probs = jax.vmap(jnp.dot,axis_name=(0,0), out_axes=0)(log_prob, weights)
            print(np.shape(weighted_log_probs))
            summed_weighted_log_probs=jnp.sum(weighted_log_probs, axis=0) #sum over all candidates
            print(np.shape(summed_weighted_log_probs))
            print(summed_weighted_log_probs)
            #loss=1
            #print(np.shape(summed_weighted_log_probs))
            #print(np.shape(mask[:,None]))
            #a=jnp.dot(summed_weighted_log_probs ,mask[:, None])
            #b=jnp.asarray(a)
            #print(b)
            #loss=-np.sum(b) #/ jnp.sum(mask)
            #print(loss)



            #loss=new_prediction_loss(params, batch_masks[0], batch_graphs[0], batch_c[0], batch_e[0], f)
            #print(loss)

batch_number 1
[  1 281 339 352 282]


: 

: 

In [2]:
import numpy as np
a=1
print(np.shape(a))

()


In [4]:

print("Entering training loop")

for epoch in range(NUM_EPOCHS):
        print("epoch", epoch+1)
        start_time = time.time()
        counter=0
        for (batch_p, batch_ce) in train_loader:
            counter=counter+1
            print("batch_number", counter)
            #batch_masks=batch_p[0]
            #batch_graphs=batch_p[1]
            #batch_c=batch_ce[0]
            #batch_e=batch_ce[1]
            #print(len(batch_masks))
            #print(len(batch_graphs))
            #print(len(batch_c))
            #print(len(batch_e))
            #print(batch_e)
            params, opt_state = update(params, opt_state, batch_p, batch_ce, f)
            print(counter, "done")
        '''
        for (p, ce) in train_loader:
            #c=ce[0]
            #e=ce[1]
            for i in range(0,len(ce[1])):#,len(e)):
                params, opt_state = update(params, opt_state, p, ce[:][i], f)
            print(f"{batch} done")
        epoch_time = time.time() - start_time
        '''
        # train_acc = accuracy(params, train_images, train_labels)
        # test_acc = accuracy(params, test_images, test_labels)
        #print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))

        #test_acc = jnp.mean(jnp.asarray([prediction_loss(params, p.mask, p.graph, s) for (p, s) in test_data]))

        #TBD!!!

        test_acc = jnp.mean(jnp.asarray([new_prediction_loss(p, p.graph, c, f) for (p, c) in test_data]))
        
        ##

        # print("Training set accuracy {}".format(train_acc))
        print("Test set accuracy {}".format(test_acc))

    # TODO: Save the model here

Entering training loop
epoch 1
batch_number 1


: 

: 